In [109]:
import joblib

text2attr = joblib.load("trained/text2attr.pkl")
text2label = joblib.load("trained/text2label.pkl")

In [110]:
import pandas as pd

files: list[str] = [
    # "dataset/transcription/hr_bot_clear.csv",
    # "dataset/transcription/hr_bot_noise.csv",
    # "dataset/transcription/hr_bot_synt.csv",
    "dataset/transcription/luga.csv",
]

li = []
for filename in files:
    df = pd.read_csv(filename)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)
df = df.sample(frac=1)
df.head()

,audio_filepath,id,text,label,attribute,denoise_filepath,transcription
4,dataset/luga/15_11_2023/2023_11_15__09_41_25.wav,2023_11_15__09_41_25,вышел из межвагонного пространства,7,-1,dataset/luga_denoise/15_11_2023/2023_11_15__09...,выселозмез выгоного пространство
164,dataset/luga/21_11_2023/2023_11_21__10_41_18.wav,2023_11_21__10_41_18,протянуть на двадцать шесть вагонов,10,26,dataset/luga_denoise/21_11_2023/2023_11_21__10...,протянут надзадти все свагонов
508,dataset/luga/03_07_2023/3fa77f6c-846e-11ee-869...,3fa77f6c-846e-11ee-869b-c09bf4619c03,протянуть на тринадцать вагонов,10,13,dataset/luga_denoise/03_07_2023/3fa77f6c-846e-...,протянутно тринадцать вагонот
18,dataset/luga/03_07_2023/3fba2d30-846e-11ee-8d3...,3fba2d30-846e-11ee-8d39-c09bf4619c03,осадить на три вагона,4,3,dataset/luga_denoise/03_07_2023/3fba2d30-846e-...,осадить на три вагона
328,dataset/luga/15_11_2023/2023_11_15__10_28_46.wav,2023_11_15__10_28_46,назад с башмака,17,-1,dataset/luga_denoise/15_11_2023/2023_11_15__10...,назад воснута


In [111]:
X = df["transcription"]
y = df["text"]

In [112]:
df["label"] = text2label.predict(X)
df["attribute"] = text2attr.predict(X)

In [113]:
from label2id import parse_label
y_pred = df.apply(lambda row: parse_label(row['label'], int(row['attribute'])), axis=1)

In [114]:
from jiwer import wer
from sklearn.metrics import f1_score

f1_w = f1_score(df["text"], y_pred, average='weighted')
print(f"F1: {f1_w:.2f}")
w = wer(df["text"].to_list(), y_pred.to_list())
print(f"Wer: {w:.2f}")

F1: 0.83
Wer: 0.13
